In [1]:
import pandas as pd
import numpy as np
import yaml

In [2]:
df = pd.read_csv("../data/Motor_Vehicle_Collisions_-_Crashes.csv.gzip", compression="gzip")

/Users/charlie/opt/miniconda3/envs/b/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head()

,Unnamed: 0,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,0,10/08/2012,10:00,NaN,NaN,NaN,NaN,NaN,OCEAN AVENUE,SHORE PARKWAY,...,Unspecified,NaN,NaN,NaN,117045,OTHER,PASSENGER VEHICLE,NaN,NaN,NaN
1,1,10/30/2012,10:40,BROOKLYN,11214,40.604338,-74.006911,"(40.6043377, -74.0069109)",BATH AVENUE,BAY 17 STREET,...,Unspecified,NaN,NaN,NaN,122248,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
2,2,11/03/2012,17:00,NaN,NaN,NaN,NaN,NaN,COLUMBUS CIRCLE,BROADWAY,...,Unspecified,NaN,NaN,NaN,39140,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
3,3,10/09/2012,19:00,MANHATTAN,10128,40.785866,-73.950929,"(40.7858664, -73.9509287)",EAST 96 STREET,LEXINGTON AVENUE,...,NaN,NaN,NaN,NaN,55853,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,NaN
4,4,11/02/2012,13:41,MANHATTAN,10014,40.728605,-74.005315,"(40.7286048, -74.0053146)",WEST HOUSTON STREET,VARICK STREET,...,Unspecified,NaN,NaN,NaN,1133,PASSENGER VEHICLE,LIVERY VEHICLE,NaN,NaN,NaN


In [4]:
[x for x in df['VEHICLE TYPE CODE 1'].unique() if 'bi' in str(x).lower()]

['Bike',
 'BICYCLE',
 'Motorbike',
 'E-Bike',
 'ebike',
 'Minibike',
 'E-BIKE',
 'E-BIK',
 'E bike',
 'E-bike',
 'DIRT BIKE',
 'E-Bik',
 'CABIN',
 'EBIKE',
 'E bik',
 'E BIK',
 'MOBIL',
 'Snowmobile',
 'E- Bi',
 'Bicyc',
 'e-bik',
 'BICYC',
 'mobil',
 'Ebike']

In [5]:
[x for x in df['VEHICLE TYPE CODE 2'].unique() if 'bi' in str(x).lower()]

['BICYCLE',
 'Bike',
 'E-Bike',
 'Motorbike',
 'Minibike',
 'E-BIKE',
 'E-bike',
 'DIRT BIKE',
 'e-bike',
 'DIRTBIKE',
 'Dirt Bike',
 'motorbike',
 'MOBILE',
 'Dirt bike',
 'E BIK',
 'E-BIK',
 'ebike',
 'E-bik',
 'Mobile foo',
 'E-Bik',
 'EBIKE',
 'E Bik',
 'E bik',
 'MOBIL',
 'Snowmobile',
 'e-BIK',
 'big r',
 'E/BIK',
 'e-bik',
 'e bik',
 'e BIK']

In [6]:
[x for x in df['VEHICLE TYPE CODE 3'].unique() if 'bi' in str(x).lower()]

['BICYCLE', 'Bike', 'E-Bike', 'E-BIKE', 'Motorbike', 'E-Bik']

In [7]:
[x for x in df['VEHICLE TYPE CODE 4'].unique() if 'bi' in str(x).lower()]

['BICYCLE', 'Bike', 'E-Bike']

In [8]:
def bike_was_present(row):
    vehicles = " ".join(str(row[f"VEHICLE TYPE CODE {r}"]) for r in (1, 2, 3, 4))
    # start with whitespace to exclude motorbike
    vehicles = " " + vehicles.lower()
    return any(x in vehicles for x in (" bik", " bic", "-bik", "ebik"))

In [9]:
df = df[df.apply(bike_was_present, axis=1)]

In [10]:
# -- collect the data we use in a smaller dataframe `subdf`
orig_names = ["CRASH TIME", "LATITUDE", "LONGITUDE", "NUMBER OF PERSONS INJURED", "NUMBER OF PERSONS KILLED", "NUMBER OF CYCLIST KILLED", "NUMBER OF CYCLIST INJURED","BOROUGH", "ZIP CODE"]
new_names = ["time", "lat", "long", "injuries", "deaths", "bikedeaths", "bikeinjuries", "borough", "zip"]
subdf = df[orig_names]
subdf = subdf.rename(columns=dict(zip(orig_names, new_names)))
subdf = subdf.loc[subdf["lat"] > 0]
subdf = subdf.loc[subdf["long"] < 0]
subdf = subdf.replace(r'^\s*$', np.nan, regex=True)
def intna(x):
    try:
        return int(x)
    except:
        return np.nan
subdf["zip"] = subdf["zip"].apply(intna)
subdf = subdf.dropna()

# -- add regression variables
subdf["injury"] = subdf["injuries"] > 0
subdf["datetime"] = pd.to_datetime(subdf["time"])
six = pd.to_datetime("6:00")
sixthirty = pd.to_datetime("6:30")
ninethirty = pd.to_datetime("9:30")
seven = pd.to_datetime("19:00")
four = pd.to_datetime("16:00")
daytime = ((subdf["datetime"] > six) & (subdf["datetime"] < seven))
subdf['date'] = pd.to_datetime(df['CRASH DATE'])
subdf['dayofweek'] = subdf['date'].apply(lambda x: x.isoweekday())
weekday = subdf['dayofweek'].apply(lambda x: x in range(1,6))
subdf['weekday'] = weekday
rush_hour = (sixthirty < subdf["datetime"]) & (subdf["datetime"] < ninethirty) | (four < subdf["datetime"]) & (subdf["datetime"] < seven)
subdf["rush_hour"] = rush_hour            # backward-compatible
subdf["rush_hour2"] = rush_hour & weekday # corrected
subdf["nighttime"] = ~daytime

# -- boroughs
subdf["manhattan"] = subdf["borough"] == "MANHATTAN"
subdf["staten"] = subdf["borough"] == "STATEN ISLAND"
subdf["queens"] = subdf["borough"] == "QUEENS"
subdf["brooklyn"] = subdf["borough"] == "BROOKLYN"
subdf["bronx"] = subdf["borough"] == "BRONX"
subdf["boro"] = subdf["borough"].astype('category').cat.codes + 1

In [11]:
len(subdf)

38938

In [12]:
# means
print(subdf.groupby(['weekday','nighttime']).injuries.mean().unstack().T)

weekday       False     True 
nighttime                    
False      0.805896  0.806550
True       0.829294  0.815921


In [13]:
# standard error (assuming independence)
subdf.groupby(['weekday','nighttime']).injuries.std().unstack().T / subdf.groupby(['weekday','nighttime']).injuries.count().unstack().T**0.5

weekday,False,True
nighttime,,
False,0.005805,0.003390
True,0.007135,0.004508


In [14]:
# means
subdf.groupby(['rush_hour','nighttime']).injuries.mean().unstack().T

rush_hour,False,True
nighttime,,
False,0.798704,0.814988
True,0.819765,NaN


In [15]:
# means
subdf.groupby(['weekday','rush_hour']).injuries.count().unstack().T

weekday,False,True
rush_hour,,
False,7329,19453
True,2505,9651


In [16]:
subdf.head()

,time,lat,long,injuries,deaths,bikedeaths,bikeinjuries,borough,zip,injury,...,weekday,rush_hour,rush_hour2,nighttime,manhattan,staten,queens,brooklyn,bronx,boro
69,15:29,40.617945,-74.033311,1.0,0.0,0,1,BROOKLYN,11209.0,True,...,True,False,False,False,False,False,False,True,False,2
78,16:20,40.760928,-73.969133,0.0,0.0,0,0,MANHATTAN,10022.0,False,...,False,True,False,False,True,False,False,False,False,3
87,12:30,40.827812,-73.925929,1.0,0.0,0,1,BRONX,10451.0,True,...,False,False,False,False,False,False,False,False,True,1
97,20:17,40.719585,-73.946991,0.0,0.0,0,0,BROOKLYN,11222.0,False,...,False,False,False,True,False,False,False,True,False,2
107,17:56,40.649522,-73.955789,1.0,0.0,0,1,BROOKLYN,11226.0,True,...,True,True,True,False,False,False,False,True,False,2


In [17]:
(subdf["injuries"] < subdf["bikeinjuries"]).mean()

5.136370640505419e-05

In [18]:
(subdf["injuries"] == subdf["bikeinjuries"]).mean()

0.9611433561045765

we can see that almost all injuries in bike-involved crashes are on the part of the cyclist

In [19]:
(subdf["deaths"] == subdf["bikedeaths"]).mean()

0.9998715907339873

In [20]:
# -- zip code processing
children = yaml.safe_load(open("../data/children.yaml", "r"))
adj = yaml.safe_load(open("../data/parentadj.yaml", "r"))

In [21]:
parents = {child: parent for parent, cs in children.items() for child in cs}

In [22]:
subdf["pzip"] = [parents[int(z)] if int(z) in parents else z for z in subdf["zip"]]

In [23]:
allnodes = set([v for a, b in adj.items() for v in (a, *b)])

In [24]:
validzip = [z in allnodes for z in subdf["pzip"]]

In [25]:
np.mean(validzip)

0.9897529405721917

In [26]:
subdf = subdf[validzip]

In [27]:
subdf.to_csv("../data/bikedf.csv.gz")